In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="2039163698@qq.com", timeout=10)

# Read your CSV file
data = pd.read_csv('D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_1.csv')

# Create a function to handle address geocoding with retries
def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  # Wait a few seconds between retries
    return None, None

# Batch processing function
def batch_geocode(data, batch_size=100, sleep_time=1):
    latitudes = []
    longitudes = []
    
    # Process data in batches
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        print(f"Processing batch {i // batch_size + 1}...")
        
        for address in batch['address']:
            lat, lon = geocode_address(address)
            latitudes.append(lat)
            longitudes.append(lon)
            sleep(sleep_time)  # Pause between each request to respect Nominatim's rate limits
        
        print(f"Batch {i // batch_size + 1} completed.")
    
    return latitudes, longitudes

# Perform batch processing, assuming each batch processes 100 records
batch_size = 100  # Process 100 addresses per batch
latitudes, longitudes = batch_geocode(data, batch_size=batch_size, sleep_time=2)

# Add geocoding results to the dataset
data['latitude'] = latitudes
data['longitude'] = longitudes

# Save all processed results to a CSV file, regardless of success
csv_output_path = 'D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_addresses_to_lat_lng.csv'
data.to_csv(csv_output_path, index=False)

print(f"All address data has been saved to {csv_output_path}")


In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
import re
from time import sleep

# Initialize the Nominatim geocoder
geolocator = Nominatim(user_agent="2039163698@qq.com", timeout=10)

# Read the CSV file
data = pd.read_csv('D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_addresses_to_lat_lng.csv')

# Function to handle address geocoding with retries
def geocode_address(address, retries=3):
    for attempt in range(retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return location.latitude, location.longitude
            else:
                return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying ({attempt+1}/{retries})...")
            sleep(2)  # Wait 2 seconds between retries
    return None, None

# Remove numbers and symbols from addresses
def remove_numbers(address):
    return re.sub(r'[^a-zA-Z\s]', '', address).strip()

# Filter addresses with missing latitude and longitude
no_lat_lng = data[data['latitude'].isna() | data['longitude'].isna()].copy()

# Clean addresses by removing numbers and symbols
no_lat_lng['cleaned_address'] = no_lat_lng['address'].apply(remove_numbers)

# Geocode the cleaned addresses
no_lat_lng['latitude'], no_lat_lng['longitude'] = zip(*no_lat_lng['cleaned_address'].apply(geocode_address))

# Update the original dataset with the new latitude and longitude values
data.update(no_lat_lng[['latitude', 'longitude']])

# Save all results back to the CSV file
csv_output_path = 'D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_addresses_to_lat_lng.csv'
data.to_csv(csv_output_path, index=False)

print(f"All address data has been saved to {csv_output_path}")


In [ ]:
import pandas as pd

# Load your dataset
file_path = 'D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_addresses_to_lat_lng.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Drop rows that contain NaN values in 'latitude' or 'longitude'
cleaned_data = data.dropna(subset=['latitude', 'longitude'])

# Save the cleaned data to a new CSV file
output_file_path = 'D:\STUDYfile\\ads2\\111ads\project-2-group-real-estate-industry-project-35\data\curated\property\property_addresses_to_lat_lng.csv'  # Replace with your desired save path
cleaned_data.to_csv(output_file_path, index=False)

print("Rows with missing latitude or longitude have been removed, and the cleaned data has been saved.")
